In [1]:
import pandas as pd
import numpy as np  

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df=pd.read_csv("/kaggle/input/piramal/dataset/test_main_loan.csv")

In [4]:
mode_value = df['OCCUPATION_TYPE'].mode()[0]
df['OCCUPATION_TYPE'].fillna(value=mode_value, inplace=True)
df.loc[df['OCCUPATION_TYPE'] == "R18", 'OCCUPATION_TYPE'] = "SALARIED"
unique_values = df['OCCUPATION_TYPE'].unique()
unique_values

/tmp/ipykernel_33/486089511.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['OCCUPATION_TYPE'].fillna(value=mode_value, inplace=True)


array(['SEP', 'SALARIED', 'SENP', 'OTHERS', 'Salaried'], dtype=object)

In [5]:
df['OCCUPATION_TYPE'] = df['OCCUPATION_TYPE'].replace("Salaried", "SALARIED")

In [6]:
df = pd.get_dummies(df, columns=['GENDER', 'OWNERSHIP_TYPE',"ACCOUNT_TYPE","OCCUPATION_TYPE"])
df= df.astype(int, errors='ignore')
df['DATE_OF_BIRTH'] = pd.to_datetime(df['DATE_OF_BIRTH'], format='%d-%m-%Y')
df['DATE_OPENED'] = pd.to_datetime(df['DATE_OPENED'], format='%d-%m-%Y')
df['age of person'] = (df['DATE_OPENED'] - df['DATE_OF_BIRTH']).dt.days / 365.25
df['AMOUNT_OVERDUE'].fillna(0, inplace=True)
df["total amount"]=df["HIGH_CREDIT_OR_SANCTIONED_AMOUNT"]-df["CURRENT_BALANCE"]+df["AMOUNT_OVERDUE"]
df['Reported_Date'] = pd.to_datetime(df['Reported_Date'], format='%d-%m-%Y')
df["date_opened_reported_date"]=(df['DATE_OPENED'] - df['Reported_Date']).dt.days / 365.25
df = df.drop(['DATE_OPENED','PAYMENT_HISTORY_START_DATE','Reported_Date','DATE_OF_BIRTH',"DATE_OF_LAST_PAYMENT","DATE_REPORTED_AND_CERTIFIED","PAYMENT_HISTORY_END_DATE"], axis=1)

/tmp/ipykernel_33/3817521009.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['AMOUNT_OVERDUE'].fillna(0, inplace=True)


In [7]:
history1=pd.read_csv("/kaggle/input/test-data/newdftest/history1")
history2=pd.read_csv("/kaggle/input/test-data/newdftest/history2")

In [8]:
def process_payment_history(entry):
    # Check if the entry is not a string (including NaN/None which are float in pandas)
    if not isinstance(entry, str):
        return float('nan'), 0, 0, 0, 0, 0, 0
    # Replacement rules
    replacements = {'STD': '000', 'SMA': '060', 'SUB': '090', 'DBT': '090', 'LSS': '090', 'XXX': 'XXX'}
    
    # Replace according to rules
    for key, value in replacements.items():
        entry = entry.replace(key, value)
    # Initialize counts and total for average calculation
    count_xxx = entry.count('XXX')
    nums = []  
    i = 0
    zero=0
    zero_30=0
    thirty_60=0
    sixty_90=0
    ninety_plus=0
    while i < len(entry):
        group = entry[i:i+3]  

        if 'X' in group:
            number_before_x = ''.join([char for char in group if char.isdigit()])
            if number_before_x:  
                nums.append(int(number_before_x))

            i += group.find('X')  
            while i < len(entry) and entry[i] == 'X':
                i += 1  # Skip the 'X's
        else:
            try:
                if group!="060" and group!="090":
                    nums.append(int(group))
            except ValueError:
                # This catches cases where conversion to integer might fail
                pass
            i += 3  # Move to the next group
    
    for i in range(0,len(nums)):
        num=nums[i]
        if num == 0:
            zero += 1
        elif 0 < num <= 30:
            zero_30 += 1
        elif 30 < num <= 60:
            # Check if the entry is not a string (including NaN/None which are float in pandas)
            thirty_60 += 1
        elif 60 < num <= 90:
            sixty_90 += 1
        elif num > 90:
            ninety_plus += 1
    avg_nums = sum(nums)
    if len(nums)==0:
        t=0
    else:
        t= nums[len(nums)-1]
    if len(nums)==0:
        t1=0
    else:
        t1= nums[0]
    return avg_nums, count_xxx,zero, zero_30 , thirty_60 , sixty_90,ninety_plus, t,t1


In [9]:
# Create a mask for rows in df where PAYMENT_HISTORY_1 contains 'E'
mask = df['PAYMENT_HISTORY_1'].str.contains('E')

# Create a mapping series from history_1
mapping_series = history1.set_index('ID')['PAYMENT_HISTORY_1']

# Replace values in df where mask is True by mapping using the 'ID' column
df.loc[mask, 'PAYMENT_HISTORY_1'] = df.loc[mask, 'ID'].map(mapping_series)
# Create a mask for rows in df where PAYMENT_HISTORY_1 contains 'E'
mask = df['PAYMENT_HISTORY_2'].notnull() & df['PAYMENT_HISTORY_2'].str.contains('E')

# Create a mapping series from history_1
mapping_series = history2.set_index('ID')['PAYMENT_HISTORY_2']

# Replace values in df where mask is True by mapping using the 'ID' column
df.loc[mask, 'PAYMENT_HISTORY_2'] = df.loc[mask, 'ID'].map(mapping_series)
def convert_sci_notation(s):
    try:
        # Convert to float, then to integer, and finally to string to exclude the decimal part
        return '{:.0f}'.format(float(s))
    except ValueError:
        # Return the original string if it cannot be converted to float
        return s
df['PAYMENT_HISTORY_1'] = df['PAYMENT_HISTORY_1'].apply(convert_sci_notation)
df['PAYMENT_HISTORY_2'] = df['PAYMENT_HISTORY_2'].apply(convert_sci_notation)
# Apply the function to the PAYMENT_HISTORY_1 column
results1 = df['PAYMENT_HISTORY_1'].apply(process_payment_history)

# Split the results into separate columns
df['avg_nums1'], df['count_xxx1'], df['zero1'], df['zero_301'], df['thirty_601'], df['sixty_901'], df['ninety_plus1'], df['firstlaste1'] ,df['lastlate1']= zip(*results1)

# Apply the function to the PAYMENT_HISTORY_2 column
results2 = df['PAYMENT_HISTORY_2'].apply(process_payment_history)

# Split the results into separate columns
df['avg_nums2'], df['count_xxx2'], df['zero2'], df['zero_302'], df['thirty_602'], df['sixty_902'], df['ninety_plus2'], df['firstlaste2'] ,df['lastlate2'] = zip(*results2)

# Calculate combined averages and counts
df['avg_nums_combined'] = (df['avg_nums1'] + df['avg_nums2']) / 36
df['count_xxx_combined'] = df['count_xxx1'] + df['count_xxx2']
df['zero_combined'] = df['zero1'] + df['zero2']
df['zero_30_combined'] = df['zero_301'] + df['zero_302']
df['thirty_60_combined'] = df['thirty_601'] + df['thirty_602']
df['sixty_90_combined'] = df['sixty_901'] + df['sixty_902']
df['ninety_plus_combined'] = df['ninety_plus1'] + df['ninety_plus2']
df['firstlaste1 combined']=df['firstlaste2']
df['lastlate1']=df['lastlate1']
# Drop the intermediate columns
df = df.drop(columns=['avg_nums1', 'count_xxx1', 'zero1', 'zero_301', 'thirty_601', 'sixty_901', 'ninety_plus1',
                      'avg_nums2', 'count_xxx2', 'zero2', 'zero_302', 'thirty_602', 'sixty_902', 'ninety_plus2',
                      'PAYMENT_HISTORY_1', 'PAYMENT_HISTORY_2','firstlaste1','firstlaste2','lastlate1','lastlate2'])


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29983 entries, 0 to 29982
Data columns (total 34 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                29983 non-null  object 
 1   HIGH_CREDIT_OR_SANCTIONED_AMOUNT  29983 non-null  int64  
 2   CURRENT_BALANCE                   29983 non-null  int64  
 3   ACTUAL_PAYMT_AMT                  28189 non-null  float64
 4   EMI_AMOUNT                        8538 non-null   float64
 5   REPAYMENT_TENURE                  14246 non-null  float64
 6   LOAN_CLASSIFICATION               29983 non-null  int64  
 7   AMOUNT_OVERDUE                    29983 non-null  float64
 8   COLLATERALVALUE                   20345 non-null  float64
 9   TU_SCORE                          29983 non-null  int64  
 10  GENDER_Female                     29983 non-null  int64  
 11  GENDER_Male                       29983 non-null  int64  
 12  GEND

In [11]:
y=df["ID"].tolist()

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()



# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Select only the numeric columns for normalization
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Apply the scaler to the DataFrame
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [13]:
df.isnull().sum()

ID                                      0
HIGH_CREDIT_OR_SANCTIONED_AMOUNT        0
CURRENT_BALANCE                         0
ACTUAL_PAYMT_AMT                     1794
EMI_AMOUNT                          21445
REPAYMENT_TENURE                    15737
LOAN_CLASSIFICATION                     0
AMOUNT_OVERDUE                          0
COLLATERALVALUE                      9638
TU_SCORE                                0
GENDER_Female                           0
GENDER_Male                             0
GENDER_Other                            0
OWNERSHIP_TYPE_Individual               0
OWNERSHIP_TYPE_Joint                    0
ACCOUNT_TYPE_Business Loan              0
ACCOUNT_TYPE_Housing Loan               0
ACCOUNT_TYPE_Personal Loan              0
ACCOUNT_TYPE_Property Loan              0
OCCUPATION_TYPE_OTHERS                  0
OCCUPATION_TYPE_SALARIED                0
OCCUPATION_TYPE_SENP                    0
OCCUPATION_TYPE_SEP                     0
age of person                     

In [14]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [15]:
imputer = IterativeImputer(max_iter=10, random_state=0)

In [16]:
selected_columns = [ 'ACTUAL_PAYMT_AMT', 'EMI_AMOUNT', 'COLLATERALVALUE','REPAYMENT_TENURE','HIGH_CREDIT_OR_SANCTIONED_AMOUNT','CURRENT_BALANCE','OWNERSHIP_TYPE_Individual','OWNERSHIP_TYPE_Joint','ACCOUNT_TYPE_Personal Loan']

In [17]:
imputed_data = imputer.fit_transform(df[selected_columns])

/opt/conda/lib/python3.10/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [18]:
imputed_df = pd.DataFrame(imputed_data, columns=selected_columns)

In [19]:
df[selected_columns] = imputed_df

In [20]:
df.isnull().sum()

ID                                  0
HIGH_CREDIT_OR_SANCTIONED_AMOUNT    0
CURRENT_BALANCE                     0
ACTUAL_PAYMT_AMT                    0
EMI_AMOUNT                          0
REPAYMENT_TENURE                    0
LOAN_CLASSIFICATION                 0
AMOUNT_OVERDUE                      0
COLLATERALVALUE                     0
TU_SCORE                            0
GENDER_Female                       0
GENDER_Male                         0
GENDER_Other                        0
OWNERSHIP_TYPE_Individual           0
OWNERSHIP_TYPE_Joint                0
ACCOUNT_TYPE_Business Loan          0
ACCOUNT_TYPE_Housing Loan           0
ACCOUNT_TYPE_Personal Loan          0
ACCOUNT_TYPE_Property Loan          0
OCCUPATION_TYPE_OTHERS              0
OCCUPATION_TYPE_SALARIED            0
OCCUPATION_TYPE_SENP                0
OCCUPATION_TYPE_SEP                 0
age of person                       0
total amount                        0
date_opened_reported_date           0
avg_nums_com

In [21]:
df.shape

(29983, 34)

In [22]:
df_id = df[['ID']]

In [23]:
df_id.shape

(29983, 1)

In [24]:
df_Test = df.drop(columns=["ID"])

In [25]:
df_Test.head(10)

,HIGH_CREDIT_OR_SANCTIONED_AMOUNT,CURRENT_BALANCE,ACTUAL_PAYMT_AMT,EMI_AMOUNT,REPAYMENT_TENURE,LOAN_CLASSIFICATION,AMOUNT_OVERDUE,COLLATERALVALUE,TU_SCORE,GENDER_Female,GENDER_Male,GENDER_Other,OWNERSHIP_TYPE_Individual,OWNERSHIP_TYPE_Joint,ACCOUNT_TYPE_Business Loan,ACCOUNT_TYPE_Housing Loan,ACCOUNT_TYPE_Personal Loan,ACCOUNT_TYPE_Property Loan,OCCUPATION_TYPE_OTHERS,OCCUPATION_TYPE_SALARIED,OCCUPATION_TYPE_SENP,OCCUPATION_TYPE_SEP,age of person,total amount,date_opened_reported_date,avg_nums_combined,count_xxx_combined,zero_combined,zero_30_combined,thirty_60_combined,sixty_90_combined,ninety_plus_combined,firstlaste1 combined
0,0.015154,0.012912,0.003317,0.013440,0.198887,0.0,0.000000,0.004883,0.786667,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.144460,0.150242,0.638757,0.000000,0.033333,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.006834,0.005795,0.004367,0.014978,0.228996,0.0,0.000000,0.002355,0.766667,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.400431,0.143379,0.942865,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.006372,0.006427,0.003641,0.012077,0.237162,0.0,0.000000,0.003344,0.883333,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.589880,0.138892,0.984597,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.009554,0.009677,0.001886,0.008213,0.351878,0.0,0.000000,0.003776,0.716667,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.444205,0.139409,0.725645,0.000000,0.033333,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.004823,0.005080,0.003226,0.010884,0.240299,0.0,0.000000,0.001521,0.623333,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.466915,0.137708,0.992364,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.016891,0.018559,0.002660,0.011853,0.471488,1.0,0.027638,0.003782,0.486667,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.195097,0.158293,0.739205,0.792381,0.033333,0.000000,0.058824,0.055556,0.058824,0.914286,0.951613
6,0.005919,0.003429,0.000754,0.004783,0.258693,0.0,0.000000,0.001182,0.750000,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.233144,0.148950,0.730121,0.000000,0.033333,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.011061,0.011681,0.003280,0.011449,0.249616,0.0,0.000000,0.004166,0.713333,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.125711,0.137797,0.973144,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.026462,0.027892,0.005727,0.021217,0.273947,0.0,0.000000,0.011658,0.760000,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.374034,0.138369,0.960242,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.018733,0.019673,0.003921,0.013578,0.262899,0.0,0.000000,0.004818,0.863333,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.291390,0.138420,0.949447,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
from joblib import load

# Load the XGBoost model from the file
xgb_loaded = load('/kaggle/input/training-model/xgb_7.10MAPE (1).joblib')


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:15:22] WARNING: /workspace/src/gbm/gbtree.cc:385: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:15:22] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


In [27]:
pred = xgb_loaded.predict(df_Test)

In [31]:
len(pred)

29983

In [34]:
df_id=df_id["ID"]
df_id=df_id.tolist()

In [35]:
predictions = pd.DataFrame({'ID':df_id, 'ACTUAL_ROI':pred})

In [36]:
predictions[:6]

,ID,ACTUAL_ROI
0,A000614177,11.530889
1,A001744326,17.671827
2,A000242802,10.878709
3,B1000244547,11.253797
4,B3000178046,18.402325
5,A000603532,11.483684


In [37]:
predictions.to_csv("PREDICTIONS.csv", index=False)
